In [1]:
import setup
setup.init_django()

In [2]:
from market.models import StockQuote

In [3]:
from django.db.models import Avg, F, RowRange, Window, Max
from django.db.models.functions import TruncDate
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal

In [4]:
days_ago = 30
now = timezone.now()
start_date = now - timedelta(days=30)
end_date = now

qs = StockQuote.objects.filter(time__range=(start_date, end_date))
qs.count()

19214

In [5]:
total = 0
for obj in qs:
    total += obj.close_price

In [6]:
total / qs.count()

Decimal('301.3717889143332986364109503')

In [7]:
qs.values('company').annotate(avg_price=Avg('close_price'))

<QuerySet [{'company': 3, 'avg_price': Decimal('578.0424125591757171')}, {'company': 2, 'avg_price': Decimal('172.2664964696223317')}, {'company': 7, 'avg_price': Decimal('420.9168394362416107')}, {'company': 1, 'avg_price': Decimal('229.0419005203171457')}, {'company': 8, 'avg_price': Decimal('140.9261248098859316')}]>

In [8]:
count = 5
ticker = "AAPL"
rolling_qs = list(qs.filter(company__ticker=ticker).order_by('-time')[:count])
rolling_qs.reverse()

In [9]:
rolling_qs

[<StockQuote: StockQuote object (8621061)>,
 <StockQuote: StockQuote object (8621062)>,
 <StockQuote: StockQuote object (8621063)>,
 <StockQuote: StockQuote object (8621064)>,
 <StockQuote: StockQuote object (8621065)>]

In [10]:
total = 0
for i, obj in enumerate(rolling_qs):
    total += obj.close_price
    avg = total / (i + 1)
    print(i + 1, obj.id, obj.close_price, avg)

1 8621061 225.1500 225.1500
2 8621062 225.0200 225.0850
3 8621063 224.9800 225.0500
4 8621064 225.0000 225.0375
5 8621065 225.0000 225.0300


In [33]:
lastest_daily_timestamps = (
    StockQuote.objects.filter(company__ticker=ticker, time__range=(start_date - timedelta(days=40), end_date))
    .annotate(date=TruncDate('time'))
    .values('company', 'date')
    .annotate(latest_time=Max('time'))
    .values('company', 'date', 'latest_time')
    .order_by('date')
)

acutal_timestamps = [x['latest_time'] for x in lastest_daily_timestamps]

In [34]:
qs = StockQuote.objects.filter(
    company__ticker=ticker, 
    time__range=(start_date, end_date),
    time__in=acutal_timestamps
)

In [35]:
frame_start = -(count - 1)
ma_val = qs.annotate(
        ma=Window(
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame=RowRange(start=frame_start, end=0),
        )
).order_by('-time')

In [36]:
for obj in ma_val[:5]:
    print(obj.id, obj.close_price, obj.ma, obj.time)

8621065 225.0000 225.4460000000000000 2024-11-13 23:25:00+00:00
6476367 224.2000 225.9420200000000000 2024-11-12 23:50:00+00:00
6476181 224.1100 225.6320200000000000 2024-11-11 23:55:00+00:00
6503221 226.9600 225.4020200000000000 2024-11-09 00:55:00+00:00
6503204 226.9600 224.3500200000000000 2024-11-08 23:55:00+00:00


In [37]:
frame_start = -(count - 1)
ma_vals = qs.annotate(
        ma_5=Window(
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame=RowRange(start=-4, end=0),
        ),
        ma_20=Window(
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame=RowRange(start=-19, end=0),
        )
).order_by('-time').first()

ma_vals.id, ma_vals.close_price, ma_vals.ma_5, ma_vals.ma_20

(8621065,
 Decimal('225.0000'),
 Decimal('225.4460000000000000'),
 Decimal('228.0193950000000000'))